# OLI Torus XAPI ETL Pipeline - XAPI Events Processing

This notebook provides an interface for bulk processing of historical XAPI data using AWS Lambda.

### Supported Event Types:
- `video` - Video interactions (play, pause, seek, complete)
- `activity_attempt` - Activity completion events  
- `page_attempt` - Page completion events
- `page_viewed` - Page view tracking
- `part_attempt` - Individual question responses

## S3 ETL Performance

The ETL pipeline uses **pure ClickHouse S3 ingestion** for all operations:

- **ClickHouse S3 Ingestion**: Direct processing using ClickHouse's native `s3()` table functions
- **Unified Processing**: Single high-performance SQL query for all event types
- **No Timeout Limits**: Bypasses Lambda's 15-minute limitation completely
- **Consistent Performance**: Same fast processing for any dataset size
- **Cost Effective**: Minimal Lambda execution time - ClickHouse does the heavy lifting

## Performance Benefits

- **10x Faster Processing**: ClickHouse S3 integration + unified table provides significant performance improvements
- **No Lambda Timeout Limits**: Process unlimited datasets without time constraints
- **Simplified Queries**: Single table queries are much faster than multi-table JOINs
- **Better Indexing**: LowCardinality event_type field enables efficient filtering and grouping

## Prerequisites

1. **AWS Configuration**: AWS credentials with Lambda invoke and S3 access permissions
2. **ClickHouse Access**: ClickHouse instance with S3 access and `raw_events` table created
3. **Lambda Deployment**: Updated Lambda functions deployed with unified table support
4. **Environment Setup**: Proper .env configuration for AWS and ClickHouse credentials

## Compatibility

- ✅ **Google Colab**: All dependencies installable via pip, no local file dependencies
- ✅ **Local Jupyter**: Full functionality including direct local execution mode

## Quick Setup - Install Requirements

Run the cell below to install all required dependencies for this notebook:

In [ ]:
# Install all required dependencies
# Choose one of the options below:

# Option 1: Install notebook-specific requirements (includes pandas and visualization tools)
!pip install -r notebook-requirements.txt

In [ ]:
# Load Environment Variables using python-dotenv
import os
import boto3
import json
import pandas as pd
from datetime import datetime, timedelta

# Install python-dotenv if not already installed
from dotenv import load_dotenv

print("🔧 Loading environment variables from .env file...")

# Load .env file - this is much simpler than manual parsing!
if load_dotenv('.env'):
    print("✅ Environment variables loaded successfully!")

    # Check AWS configuration
    AWS_REGION = os.getenv('AWS_REGION', 'us-east-1')
    AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
    AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
    S3_XAPI_BUCKET = os.getenv('S3_XAPI_BUCKET')

    print(f"\n🔍 AWS Configuration Status:")
    print(f"  AWS_ACCESS_KEY_ID: {'✅ Set' if AWS_ACCESS_KEY_ID else '❌ Missing'}")
    print(f"  AWS_SECRET_ACCESS_KEY: {'✅ Set' if AWS_SECRET_ACCESS_KEY else '❌ Missing'}")
    print(f"  AWS_REGION: {AWS_REGION if AWS_REGION else '❌ Missing'}")
    print(f"  S3_XAPI_BUCKET: {S3_XAPI_BUCKET if S3_XAPI_BUCKET else '❌ Missing'}")

    # Initialize AWS clients
    lambda_client = boto3.client('lambda', region_name=AWS_REGION)
    s3_client = boto3.client('s3', region_name=AWS_REGION)

    ENVIRONMENT = os.getenv('ENVIRONMENT', 'dev')

    # Unified Lambda function name (adjust based on your deployment)
    XAPI_ETL_FUNCTION = f'xapi-etl-processor-{ENVIRONMENT}'

    print(f"Configured for {ENVIRONMENT} environment in {AWS_REGION}")
    print(f"Unified XAPI ETL Function: {XAPI_ETL_FUNCTION}")

    EXECUTION_MODE = os.getenv('EXECUTION_MODE', 'local').lower()

    # For local execution, we'll import the required modules
    if EXECUTION_MODE == 'local':
        import sys
        import os

        # Add current directory to path to import local modules
        current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else os.getcwd()))
        if current_dir not in sys.path:
            sys.path.insert(0, current_dir)

        # Import local modules (using unified versions)
        try:
            from lambda_function import lambda_handler, health_check
            from common import get_config
            from clickhouse_client import ClickHouseClient
            print(f"✅ Local unified modules imported successfully")
            print(f"   - Using unified raw_events table model")
            print(f"   - All event types processed into single table")
        except ImportError as e:
            print(f"❌ Failed to import local modules: {e}")
            print("Make sure you're running this notebook from the xapi-etl-processor directory")
            print("Exiting due to failed local module imports.")
            sys.exit(1)

    print(f"🔧 Execution mode: {EXECUTION_MODE.upper()}")
    if EXECUTION_MODE == 'lambda':
        print(f"   Using Lambda function: {XAPI_ETL_FUNCTION}")
        print(f"   📊 Lambda processes via unified raw_events table")
    else:
        print(f"   Running locally with unified table model")
        print(f"   📊 Direct ClickHouse processing into raw_events table")

else:
    print(f"❌ Could not load .env file")
    print(f"   Make sure .env file exists with your AWS credentials:")
    print(f"   cp example.env .env")
    print(f"   # Then edit .env with your actual credentials")

## Execution Modes

This notebook supports two execution modes:

### Local Mode (Default)
- **EXECUTION_MODE = 'local'**
- Runs the Lambda function code locally in this notebook
- Requires local modules (lambda_function.py, common.py, clickhouse_client.py)
- Good for testing and development
- All operations run synchronously in the notebook

### Lambda Mode
- **EXECUTION_MODE = 'lambda'**
- Invokes the deployed AWS Lambda function remotely
- Requires AWS credentials and deployed Lambda function
- Best for production use and processing large datasets
- Supports true asynchronous processing

**To switch modes:** Change the `EXECUTION_MODE` variable in the cell above and re-run that cell.

## Clean API Architecture

This notebook uses a **clean, optimized API** with **unified section processing**:

- **All processing logic** resides in the `lambda_function.py`
- **Notebook is just a caller** - no complex processing logic here
- **Unified section processing** - single or multiple sections use the same code path
- **Clean API** - only `section_ids` parameter (always a list)
- **Consistent interface** regardless of local or Lambda execution mode
- **Lambda function handles**:
  - Single file processing (S3 event triggers)
  - Unified bulk processing (`section_ids: [id]` for single, `section_ids: [id1, id2, ...]` for multiple)
  - Automatic batching for multiple sections
  - Error handling and retries
  - Progress tracking and reporting

### Benefits of This Approach

1. **Single Source of Truth**: All logic in `lambda_function.py`
2. **Unified Code Path**: Single or multiple sections use identical processing logic
3. **Clean API**: No deprecated parameters, consistent interface
4. **Easy Testing**: Test locally before deploying to Lambda
5. **Consistent Behavior**: Same results whether run locally or remotely
6. **Simplified Debugging**: All processing logic in one place
7. **Easy Deployment**: Just deploy the Lambda function, notebook stays simple
8. **Flexible Scaling**: Single section is just a list with one element

### API Design

- **Single Section**: `section_ids: [123]` 
- **Multiple Sections**: `section_ids: [123, 124, 125]`
- **Required Parameters**: `mode: "bulk"`, `section_ids: [...]`
- **Optional Parameters**: `batch_size`, `force_reprocess`, `dry_run`, `start_date`, `end_date`, `s3_bucket`, `s3_prefix`

## Helper Functions

In [ ]:
def execute_function(payload):
    """Execute function either locally or via Lambda based on EXECUTION_MODE"""
    if EXECUTION_MODE == 'local':
        return execute_local(payload)
    else:
        return invoke_lambda_sync(payload)

def execute_function_async(payload):
    """Execute function asynchronously - Lambda only or simulate locally"""
    if EXECUTION_MODE == 'local':
        # For local execution, run synchronously but indicate it's "async"
        result = execute_local(payload)
        if result['success']:
            result['async_mode'] = True
            result['request_id'] = f"local-{datetime.now().strftime('%Y%m%d%H%M%S')}"
        return result
    else:
        return invoke_lambda_async(payload)

def execute_local(payload):
    """Execute the Lambda function locally"""
    try:
        # Create a mock context object
        class MockContext:
            def __init__(self):
                self.function_name = "xapi-etl-processor-local"
                self.function_version = "1"
                self.aws_request_id = f"local-{datetime.now().strftime('%Y%m%d%H%M%S')}"

        context = MockContext()

        # Call the lambda_handler function directly
        result = lambda_handler(payload, context)

        return {
            'success': True,
            'status_code': result.get('statusCode', 200),
            'result': result,
            'execution_mode': 'local'
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'execution_mode': 'local'
        }

def invoke_lambda_async(payload):
    """Invoke the Lambda function asynchronously"""
    try:
        response = lambda_client.invoke(
            FunctionName=XAPI_ETL_FUNCTION,
            InvocationType='Event',  # Async invocation
            Payload=json.dumps(payload)
        )
        return {
            'success': True,
            'status_code': response['StatusCode'],
            'request_id': response['ResponseMetadata']['RequestId'],
            'execution_mode': 'lambda',
            'async_mode': True
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'execution_mode': 'lambda'
        }

def invoke_lambda_sync(payload):
    """Invoke the Lambda function synchronously"""
    try:
        response = lambda_client.invoke(
            FunctionName=XAPI_ETL_FUNCTION,
            InvocationType='RequestResponse',  # Sync invocation
            Payload=json.dumps(payload)
        )

        result = json.loads(response['Payload'].read().decode())
        return {
            'success': True,
            'status_code': response['StatusCode'],
            'result': result,
            'execution_mode': 'lambda'
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'execution_mode': 'lambda'
        }

def check_function_health():
    """Check if the function is healthy (Lambda or local)"""
    payload = {'health_check': True}
    return execute_function(payload)

print("✅ Helper functions loaded")
print(f"🔧 Execution mode: {EXECUTION_MODE}")
print(f"🎯 All processing logic now handled by Lambda function")
print(f"📝 Notebook acts as a simple caller interface")

## 1. Health Checks

First, let's verify that our Lambda functions and ClickHouse are healthy:

In [ ]:
# Check XAPI ETL processor health
print("Checking XAPI ETL processor health...")
health = check_function_health()
print(json.dumps(health, indent=2))

In [ ]:
# Optional: Clear existing section data before reprocessing
# Uncomment and run this cell if you want to start fresh for this section

clear_section_data = False  # Set to True to enable clearing
target_section_id = '145'   # Make sure this matches your section ID above

if clear_section_data:
    print(f"⚠️  WARNING: This will delete ALL existing data for section {target_section_id}")
    print(f"   🗃️  From unified raw_events table (all event types)")

    if EXECUTION_MODE == 'local':
        try:
            # For local execution, directly use ClickHouse client
            from clickhouse_client import ClickHouseClient
            clickhouse_client = ClickHouseClient()

            # Delete all events for this section from the unified table
            print(f"🧹 Clearing unified raw_events table for section {target_section_id}...")

            try:
                # Get current count before deletion
                current_count = clickhouse_client.get_section_event_count(int(target_section_id))
                print(f"   📊 Current events in section: {current_count}")

                if current_count > 0:
                    # Delete all events for this section
                    result = clickhouse_client.delete_section_events(int(target_section_id))
                    print(f"✅ Cleared all events for section {target_section_id} from raw_events table")

                    # Verify deletion
                    remaining_count = clickhouse_client.get_section_event_count(int(target_section_id))
                    print(f"   📊 Remaining events: {remaining_count}")
                else:
                    print(f"   💡 No existing events found for section {target_section_id}")

            except Exception as e:
                print(f"⚠️  Error clearing section data: {str(e)}")

            print(f"✅ Section {target_section_id} data cleared - you can now set force_reprocess=False")

        except Exception as e:
            print(f"❌ Failed to clear section data: {str(e)}")
    else:
        print("❌ Data clearing only available in local mode")
        print("   For Lambda mode, use force_reprocess=True instead")
        print("   💡 The unified table approach makes data management more efficient")
else:
    print("💡 Section data clearing disabled")
    print("   Set clear_section_data = True above to enable section data clearing")
    print("   🗃️  Unified table: All event types stored in single raw_events table")
    print("   💡 Alternatively, use force_reprocess=True in processing calls to overwrite existing data")

## 3. Process Single Section

Process data for a single section using the unified bulk processing approach. Single sections are handled as a list with one element:

In [ ]:
# Process a single section using the Lambda function (unified approach)
target_section_id = 145  # Change this to your section ID
force_reprocess_single = False  # Set to True to reprocess existing data

print(f"🎯 Processing single section: {target_section_id}")
print(f"🗃️  Target: Unified raw_events table")

# Create payload for single section processing (using unified approach)
single_section_payload = {
    'mode': 'bulk',
    'section_ids': [target_section_id],  # Single section as list
    'force_reprocess': force_reprocess_single,
    's3_bucket': S3_XAPI_BUCKET,
    's3_prefix': 'section/'
}

print(f"🚀 Starting processing for section {target_section_id}...")

# Execute the function (either locally or via Lambda)
result = execute_function(single_section_payload)

if result['success']:
    result_body = result['result']['body']
    if isinstance(result_body, str):
        result_data = json.loads(result_body)
    else:
        result_data = result_body

    print("✅ Single section processing completed")

    # Display results
    processing_mode = result_data.get('processing_mode', 'unknown')
    total_events = result_data.get('total_events_processed', 0)
    files_found = result_data.get('files_found', 0)
    processing_method = result_data.get('processing_method', 'unknown')

    print(f"\n📊 Processing Results:")
    print(f"   Processing mode: {processing_mode}")
    print(f"   📁 Files found: {files_found}")
    print(f"   📈 Total events processed: {total_events}")
    print(f"   🔧 Processing method: {processing_method}")

    # Show per-section details
    section_results = result_data.get('section_results', {})
    section_result = section_results.get(str(target_section_id), {})

    if section_result:
        status = section_result.get('status', 'unknown')
        files_processed = section_result.get('files_processed', 0)

        if status == 'success':
            print(f"   ✅ Section status: {status}")
            print(f"   📄 Files processed: {files_processed}")
        elif status == 'skipped':
            reason = section_result.get('reason', 'Unknown reason')
            print(f"   ⏭️  Section status: {status} - {reason}")
        elif status == 'failed':
            error = section_result.get('error', 'Unknown error')
            print(f"   ❌ Section status: {status} - {error}")

    print(f"\n🏃 Execution mode: {result.get('execution_mode', 'unknown')}")

    if EXECUTION_MODE == 'lambda' and 'request_id' in result:
        print(f"📋 Request ID: {result['request_id']}")

    # Check if processing was skipped due to existing data
    if 'message' in result_data and 'already exists' in result_data['message']:
        existing_count = result_data.get('existing_events_count', 0)
        print(f"\n💡 Processing was skipped - section already has {existing_count} events")
        print(f"   Set force_reprocess_single = True to reprocess anyway")

else:
    print(f"❌ Single section processing failed: {result.get('error')}")

## 4. Process Multiple Sections

Process data for multiple sections using the unified bulk processing approach. The same Lambda function handles both single and multiple sections seamlessly:

In [ ]:
# Process multiple sections using the Lambda function (unified approach)
sections_to_process = [145]  # Add your section IDs here
batch_size = 5  # Process sections in batches to avoid timeouts
force_reprocess_batch = False  # Set to True to reprocess existing data

print(f"🔄 Processing {len(sections_to_process)} sections with unified approach")
print(f"📦 Batch size: {batch_size} sections per batch")
print(f"🗃️  Target: Unified raw_events table")

if sections_to_process:
    # Create payload for multiple sections processing (unified approach)
    payload = {
        'mode': 'bulk',
        'section_ids': sections_to_process,  # Works for single or multiple sections
        'batch_size': batch_size,
        'force_reprocess': force_reprocess_batch,
        's3_bucket': S3_XAPI_BUCKET,
        's3_prefix': 'section/'
    }

    print(f"🎯 Processing {len(sections_to_process)} sections...")

    # Execute the function (either locally or via Lambda)
    result = execute_function(payload)

    if result['success']:
        result_body = result['result']['body']
        if isinstance(result_body, str):
            result_data = json.loads(result_body)
        else:
            result_data = result_body

        print("✅ Sections processing completed")

        # Display summary
        processing_mode = result_data.get('processing_mode', 'unknown')
        total_sections = result_data.get('total_sections_requested', 0)
        successful_sections = result_data.get('successful_sections', 0)
        failed_sections = result_data.get('failed_sections', 0)
        total_events = result_data.get('total_events_processed', 0)

        print(f"\n📊 Processing Summary:")
        print(f"   🔧 Processing mode: {processing_mode}")
        print(f"   📋 Total sections requested: {total_sections}")
        print(f"   ✅ Successful: {successful_sections} sections")
        print(f"   ❌ Failed: {failed_sections} sections")
        print(f"   📈 Total events processed: {total_events}")

        # Show successful section IDs
        if 'successful_section_ids' in result_data:
            print(f"   🎯 Successful sections: {result_data['successful_section_ids']}")

        # Show failed section IDs
        if 'failed_section_ids' in result_data:
            print(f"   ⚠️  Failed sections: {result_data['failed_section_ids']}")

        # Show per-section details
        section_results = result_data.get('section_results', {})
        if section_results:
            print(f"\n📝 Per-Section Details:")
            for section_id, section_result in section_results.items():
                status = section_result.get('status', 'unknown')
                events = section_result.get('total_events_processed', 0)
                files_processed = section_result.get('files_processed', 0)
                files_found = section_result.get('files_found', 0)

                if status == 'success':
                    print(f"   Section {section_id}: ✅ {events} events processed ({files_processed} files)")
                elif status == 'failed':
                    error = section_result.get('error', 'Unknown error')
                    print(f"   Section {section_id}: ❌ Failed - {error}")
                elif status == 'skipped':
                    reason = section_result.get('reason', 'Unknown reason')
                    print(f"   Section {section_id}: ⏭️  Skipped - {reason} ({files_found} files found)")
                elif status == 'dry_run':
                    print(f"   Section {section_id}: 🔍 Dry run - {files_found} files found")

        print(f"\n🔧 Processing method: {result_data.get('processing_method', 'unknown')}")
        print(f"🏃 Execution mode: {result.get('execution_mode', 'unknown')}")

        if EXECUTION_MODE == 'lambda' and 'request_id' in result:
            print(f"📋 Request ID: {result['request_id']}")

    else:
        print(f"❌ Sections processing failed: {result.get('error')}")

else:
    print("💡 No sections specified for processing")
    print("   Add section IDs to sections_to_process list above")

## 5. Test Single File Processing

Test processing of a single JSONL file:

In [ ]:
# Test with a specific file using the Lambda function
test_bucket = S3_XAPI_BUCKET  # Use the configured bucket
test_key = 'section/145/video/2024-01-01T12-00-00.000Z_test-bundle.jsonl'  # Replace with actual file

print(f"🔍 Testing single file processing...")
print(f"📁 File: s3://{test_bucket}/{test_key}")

# Create test payload for single file processing
test_payload = {
    'bucket': test_bucket,
    'key': test_key
}

# Execute the function (either locally or via Lambda)
test_result = execute_function(test_payload)

if test_result['success']:
    result_body = test_result['result']['body']
    if isinstance(result_body, str):
        result_data = json.loads(result_body)
    else:
        result_data = result_body

    print("✅ Single file processing completed")

    # Display results
    events_processed = result_data.get('total_events_processed', 0)
    section_id = result_data.get('section_id', 'unknown')
    processing_method = result_data.get('processing_method', 'unknown')

    print(f"\n📊 Processing Results:")
    print(f"   📄 Section ID: {section_id}")
    print(f"   📈 Events processed: {events_processed}")
    print(f"   🔧 Processing method: {processing_method}")
    print(f"   🏃 Execution mode: {test_result.get('execution_mode', 'unknown')}")

    if EXECUTION_MODE == 'lambda' and 'request_id' in test_result:
        print(f"   📋 Request ID: {test_result['request_id']}")

else:
    print(f"❌ Single file processing failed: {test_result.get('error')}")

print(f"\n💡 Note: Make sure the test file exists in S3 before running this test")
print(f"   You can use AWS CLI or S3 console to verify: aws s3 ls s3://{test_bucket}/{test_key}")

## 6. Monitoring and Troubleshooting

Check Lambda function logs and status:

In [ ]:
# Check recent Lambda invocations (requires CloudWatch Logs access)
import boto3
from datetime import datetime, timedelta

logs_client = boto3.client('logs', region_name=AWS_REGION)

def get_recent_lambda_logs(function_name, hours=1):
    """Get recent logs for a Lambda function"""
    log_group = f'/aws/lambda/{function_name}'

    try:
        end_time = datetime.utcnow()
        start_time = end_time - timedelta(hours=hours)

        response = logs_client.filter_log_events(
            logGroupName=log_group,
            startTime=int(start_time.timestamp() * 1000),
            endTime=int(end_time.timestamp() * 1000),
            limit=100
        )

        return response.get('events', [])
    except Exception as e:
        print(f"Error getting logs for {function_name}: {str(e)}")
        return []

# Get recent logs for XAPI ETL processor
print(f"Recent logs for {XAPI_ETL_FUNCTION}:")
etl_logs = get_recent_lambda_logs(XAPI_ETL_FUNCTION)
for event in etl_logs[-5:]:  # Show last 5 log events
    timestamp = datetime.fromtimestamp(event['timestamp'] / 1000)
    print(f"[{timestamp}] {event['message']}")